In [159]:
import pandas as pd
import requests
import numpy as np
import json

df = pd.read_csv('dolar_ccl_cotizacion_historica.csv', encoding='latin1', delimiter=';', usecols=[0, 1])
df = df.rename_axis('index', axis=1)
df['fecha'] = pd.to_datetime(df['fecha'], format='%d/%m/%Y')
df['Dolar_ccl'] = pd.to_numeric(df['Dolar_ccl'], errors='coerce', downcast='float')


df

index,Dolar_ccl,fecha
0,3.910000085830688,2010-02-22
1,3.930000066757202,2010-02-26
2,3.890000104904175,2010-03-09
3,3.900000095367432,2010-03-12
4,3.910000085830688,2010-03-16
...,...,...
3246,1294.930053710937500,2024-02-05
3247,1249.420043945312500,2024-02-06
3248,1263.670043945312500,2024-02-07
3249,1246.569946289062500,2024-02-08


In [160]:
fa = pd.read_csv('factor_actualizacion.csv')
fa

,date,Infla_Diaria,Factor_Actualizacion
0,1990-01-01,1.018995330134033,9772.229412358323316
1,1990-01-02,1.018995330134033,9590.062999673351442
2,1990-01-03,1.018995330134033,9411.292393667716169
3,1990-01-04,1.018995330134033,9235.854292315358180
4,1990-01-05,1.018995330134033,9063.686573618075272
...,...,...,...
12444,2024-01-27,1.005353454232947,1.027055404269932
12445,2024-01-28,1.005353454232947,1.021586388295192
12446,2024-01-29,1.005353454232947,1.016146494542688
12447,2024-01-30,1.005353454232947,1.010735567938119


In [161]:
# Convert 'date' column in 'fa' DataFrame to datetime type
fa['date'] = pd.to_datetime(fa['date'])

# Perform inner join
result_df = pd.merge(fa, df, left_on='date', right_on='fecha', how='inner')

# Drop the redundant 'fecha' column
result_df.drop('fecha', axis=1, inplace=True)


result_df['ccl_real'] = result_df['Factor_Actualizacion'] * result_df['Dolar_ccl']
result_df['ccl_real_avg'] = result_df['ccl_real'].mean()

result_df

,date,Infla_Diaria,Factor_Actualizacion,Dolar_ccl,ccl_real,ccl_real_avg
0,2010-02-22,1.000426111147619,117.436692091345563,3.910000085830688,459.177476156833279,957.240610860945367
1,2010-02-26,1.000426111147619,117.236740805838394,3.930000066757202,460.740399193341716,957.240610860945367
2,2010-03-09,1.000352963568546,116.756905741937160,3.890000104904175,454.184375584422412,957.240610860945367
3,2010-03-12,1.000352963568546,116.633360164063561,3.900000095367432,454.870115762871876,957.240610860945367
4,2010-03-16,1.000352963568546,116.468836059158320,3.910000085830688,455.393158987909430,957.240610860945367
...,...,...,...,...,...,...
3239,2024-01-25,1.005353454232947,1.038081427338685,1281.949951171875000,1330.768435089257082,957.240610860945367
3240,2024-01-26,1.005353454232947,1.032553698371393,1231.089965820312500,1271.166497235675251,957.240610860945367
3241,2024-01-29,1.005353454232947,1.016146494542688,1263.180053710937500,1283.575983554613458,957.240610860945367
3242,2024-01-30,1.005353454232947,1.010735567938119,1269.369995117187500,1282.997402938377945,957.240610860945367


In [162]:
import plotly.express as px
fig_usd = px.line(result_df, x='date', y=['ccl_real', 'ccl_real_avg'], hover_data='Dolar_ccl', title='DOLAR CCL real')
fig_usd

In [163]:
#result_df.to_csv('datos-dolar-infla.csv', index=False)
